In [1]:
# Import libraries and packages
import pandas as pd
import numpy as np
import os
import math
import tableone
from datetime import datetime, timedelta
from scipy import stats
import matplotlib.pyplot as plt
import warnings

In [2]:
path = '/home/dchanci/research/pediatric_sepsis/prediction_ml/models/results_updated'
name = 'inf_phoenix_inf_psofa'

In [3]:
cohort1 = pd.read_csv('/labs/kamaleswaranlab/dchanci/data/pediatric_sepsis/prediction_ml/updated_data/data_screening/cohort_inf_phoenix.csv')
cohort2 = pd.read_csv('/labs/kamaleswaranlab/dchanci/data/pediatric_sepsis/prediction_ml/updated_data/data_screening/cohort_inf_psofa.csv')

In [4]:
col_names = ['albumin', 'base_excess', 'base_deficit', 'pco2', 'po2', 'bicarbonate', 'bilirubin_total', 'bp_dias', 'bp_sys', 'bun',
              'calcium', 'calcium_ionized', 'chloride', 'co2', 'creatinine', 'fio2', 'glucose', 'hemoglobin', 'lactic_acid', 
              'map', 'pao2_fio2', 'ph', 'platelets', 'potassium', 'ptt', 'pulse', 'pupil_left_size', 'resp', 'sodium', 'spo2', 'temp', 'wbc']

col_names_fixed = ['Albumin (g/dL)', 'Base Excess (mEq/L)', 'Base Deficit (mEq/L)', 'Arterial PaCO2 (mm Hg)', 'Arterial PaO2 (mm Hg)', 'Bicarbonate (mEq/L)', 'Bilirubin (mg/dL)', 
                   'Diastolic Blood Pressure (mm Hg)', 'Systolic Blood Pressure (mm Hg)', 'BUN (mg/dL)', 'Calcium (mg/dL)', 'Ionized Calcium (mg/dL)', 
                   'Chloride (mEq/L)', 'CO2 (mEq/L)', 'Creatinine (mg/dL)', 'FiO2 (%)', 'Glucose (mg/dL)', 'Hemoglobin (g/dL)', 'Lactic Acid (mEq/L)', 
                    'Mean Arterial Pressure (mm Hg)', 'PaO2/FiO2 Ratio (mmHg)', 'pH', 'Platelets (x10\u2079/L)', 'Potassium (mEq/L)', 'PTT (seconds)', 
                    'Heart Rate (beats per minute)', 'Pupil Left Size (mm)', 'Respiratory Rate (breaths per minute)', 'Sodium (mEq/L)', 'SpO2 (%)', 
                    'Temperature (°C)', 'WBC (x10\u2079/L)']

In [5]:
# Load data
dist_eg = pd.read_parquet('/labs/kamaleswaranlab/dchanci/data/pediatric_sepsis/prediction_ml/updated_data/data_analysis/features_preimp_pivot_24_eg.parquet.gzip')
dist_eg = dist_eg[(dist_eg['csn'].isin(cohort1['csn'].unique().tolist())) & (dist_eg['csn'].isin(cohort2['csn'].unique().tolist()))]
dist_eg = dist_eg[col_names]
dist_eg.columns = col_names_fixed

In [6]:
# Compute statistics
mean = [round(x,2) for x in list(np.nanmean(dist_eg, axis=0))]
mean.insert(0, "Mean")
median = [round(x,2) for x in list(np.nanmedian(dist_eg, axis=0))]
median.insert(0, "Median")
q1 = [round(x,2) for x in list(np.nanquantile(dist_eg, 0.25, axis=0))]
q1.insert(0, "Q1")
q3 = [round(x,2) for x in list(np.nanquantile(dist_eg, 0.75, axis=0))]
q3.insert(0, "Q3")
min = [round(x,2) for x in list(np.nanmin(dist_eg, axis=0))]
min.insert(0, "Min")
max = [round(x,2) for x in list(np.nanmax(dist_eg, axis=0))]
max.insert(0, "Max")
std = [round(x,2) for x in list(np.nanstd(dist_eg, axis=0))]
std.insert(0, "Std")
dist_eg.reset_index(inplace=True)
dist_eg.loc[len(dist_eg.index)] = mean
dist_eg.loc[len(dist_eg.index)] = median
dist_eg.loc[len(dist_eg.index)] = q1
dist_eg.loc[len(dist_eg.index)] = q3
dist_eg.loc[len(dist_eg.index)] = min
dist_eg.loc[len(dist_eg.index)] = max
dist_eg.loc[len(dist_eg.index)] = std
dist_eg = dist_eg.iloc[-7:]
dist_eg=dist_eg.rename(columns = {'index':'Feature'})
dist_eg = dist_eg.set_index('Feature')
dist_eg = dist_eg.T
dist_eg.reset_index(inplace=True)
dist_eg.rename(columns={'index': 'Feature'}, inplace=True)
dist_eg

Feature,Feature,Mean,Median,Q1,Q3,Min,Max,Std
0,Albumin (g/dL),2.69,2.70,2.30,3.10,1.30,4.90,0.62
1,Base Excess (mEq/L),2.90,2.00,0.00,4.00,0.00,20.00,3.47
2,Base Deficit (mEq/L),5.69,5.00,3.00,7.00,1.00,26.00,4.04
3,Arterial PaCO2 (mm Hg),44.89,42.95,36.40,51.50,17.00,85.80,12.07
4,Arterial PaO2 (mm Hg),77.61,58.00,41.00,90.00,21.00,383.00,57.05
5,Bicarbonate (mEq/L),22.41,22.30,19.60,25.10,4.80,39.20,4.89
6,Bilirubin (mg/dL),1.29,0.50,0.30,1.00,0.10,16.50,2.30
7,Diastolic Blood Pressure (mm Hg),57.39,56.00,47.00,66.00,31.00,100.00,14.22
8,Systolic Blood Pressure (mm Hg),100.28,99.00,88.00,112.00,64.00,150.00,17.35
9,BUN (mg/dL),16.99,12.00,8.00,19.00,2.00,97.00,15.45


In [7]:
# Load data
dist_sr = pd.read_parquet('/labs/kamaleswaranlab/dchanci/data/pediatric_sepsis/prediction_ml/updated_data/data_analysis/features_preimp_pivot_24_sr.parquet.gzip')
dist_sr = dist_sr[(dist_sr['csn'].isin(cohort1['csn'].unique().tolist())) & (dist_sr['csn'].isin(cohort2['csn'].unique().tolist()))]
dist_sr = dist_sr[col_names]
dist_sr.columns = col_names_fixed

In [8]:
# Compute statistics
mean = [round(x,2) for x in list(np.nanmean(dist_sr, axis=0))]
mean.insert(0, "Mean")
median = [round(x,2) for x in list(np.nanmedian(dist_sr, axis=0))]
median.insert(0, "Median")
q1 = [round(x,2) for x in list(np.nanquantile(dist_sr, 0.25, axis=0))]
q1.insert(0, "Q1")
q3 = [round(x,2) for x in list(np.nanquantile(dist_sr, 0.75, axis=0))]
q3.insert(0, "Q3")
min = [round(x,2) for x in list(np.nanmin(dist_sr, axis=0))]
min.insert(0, "Min")
max = [round(x,2) for x in list(np.nanmax(dist_sr, axis=0))]
max.insert(0, "Max")
std = [round(x,2) for x in list(np.nanstd(dist_sr, axis=0))]
std.insert(0, "Std")
dist_sr.reset_index(inplace=True)
dist_sr.loc[len(dist_sr.index)] = mean
dist_sr.loc[len(dist_sr.index)] = median
dist_sr.loc[len(dist_sr.index)] = q1
dist_sr.loc[len(dist_sr.index)] = q3
dist_sr.loc[len(dist_sr.index)] = min
dist_sr.loc[len(dist_sr.index)] = max
dist_sr.loc[len(dist_sr.index)] = std
dist_sr = dist_sr.iloc[-7:]
dist_sr=dist_sr.rename(columns = {'index':'Feature'})
dist_sr = dist_sr.set_index('Feature')
dist_sr = dist_sr.T
dist_sr.reset_index(inplace=True)
dist_sr.rename(columns={'index': 'Feature'}, inplace=True)
dist_sr

Feature,Feature,Mean,Median,Q1,Q3,Min,Max,Std
0,Albumin (g/dL),2.76,2.70,2.30,3.20,1.60,4.70,0.60
1,Base Excess (mEq/L),2.71,2.00,0.00,4.00,0.00,16.00,3.08
2,Base Deficit (mEq/L),5.60,5.00,3.00,7.00,1.00,25.50,4.06
3,Arterial PaCO2 (mm Hg),43.93,41.80,35.50,50.60,17.40,85.90,12.21
4,Arterial PaO2 (mm Hg),72.64,62.00,46.00,84.00,22.00,313.00,41.42
5,Bicarbonate (mEq/L),22.37,22.38,19.60,25.20,4.90,37.40,4.81
6,Bilirubin (mg/dL),0.85,0.40,0.20,0.70,0.10,14.70,1.61
7,Diastolic Blood Pressure (mm Hg),55.92,55.00,45.00,66.00,29.00,97.00,14.40
8,Systolic Blood Pressure (mm Hg),99.79,99.00,89.00,110.00,66.00,144.00,15.58
9,BUN (mg/dL),13.94,11.00,7.00,16.00,2.00,66.00,10.98


In [9]:
# Organize tables 
for i in range(dist_eg.shape[0]):
    dist_eg.loc[dist_eg.index == i, 'Mean (SD)'] = str(dist_eg.loc[dist_eg.index == i, 'Mean'].values[0]) + ' (' + str(dist_eg.loc[dist_eg.index == i, 'Std'].values[0]) + ')'
    dist_eg.loc[dist_eg.index == i, 'Median (Q1, Q3)'] = str(dist_eg.loc[dist_eg.index == i, 'Median'].values[0]) + ' (' + str(dist_eg.loc[dist_eg.index == i, 'Q1'].values[0]) + ', ' + str(dist_eg.loc[dist_eg.index == i, 'Q3'].values[0]) + ')'
    dist_eg.loc[dist_eg.index == i, 'Min, Max'] = str(dist_eg.loc[dist_eg.index == i, 'Min'].values[0]) + ', ' + str(dist_eg.loc[dist_eg.index == i, 'Max'].values[0])
dist_eg.drop(['Mean', 'Median', 'Q1', 'Q3', 'Min', 'Max', 'Std'], axis=1, inplace=True)


for i in range(dist_sr.shape[0]):
    dist_sr.loc[dist_sr.index == i, 'Mean (SD)'] = str(dist_sr.loc[dist_sr.index == i, 'Mean'].values[0]) + ' (' + str(dist_sr.loc[dist_sr.index == i, 'Std'].values[0]) + ')'
    dist_sr.loc[dist_sr.index == i, 'Median (Q1, Q3)'] = str(dist_sr.loc[dist_sr.index == i, 'Median'].values[0]) + ' (' + str(dist_sr.loc[dist_sr.index == i, 'Q1'].values[0]) + ', ' + str(dist_sr.loc[dist_sr.index == i, 'Q3'].values[0]) + ')'
    dist_sr.loc[dist_sr.index == i, 'Min, Max'] = str(dist_sr.loc[dist_sr.index == i, 'Min'].values[0]) + ', ' + str(dist_sr.loc[dist_sr.index == i, 'Max'].values[0])
dist_sr.drop(['Mean', 'Median', 'Q1', 'Q3', 'Min', 'Max', 'Std'], axis=1, inplace=True)
dist_sr

dist_eg = dist_eg.merge(dist_sr, on='Feature')
dist_eg.columns = [['Feature', 'Derivation', 'Derivation', 'Derivation', 'Validation', 'Validation', 'Validation'], [' ', 'Mean (SD)', 'Median (Q1, Q3)', 'Min, Max', 'Mean (SD)', 'Median (Q1, Q3)', 'Min, Max']]
dist_eg.to_csv(os.path.join(path, name + '_24_h_dist.csv'), index=False)
dist_eg

Feature      Derivation  \
                                                Mean (SD)   
0                          Albumin (g/dL)     2.69 (0.62)   
1                     Base Excess (mEq/L)      2.9 (3.47)   
2                    Base Deficit (mEq/L)     5.69 (4.04)   
3                  Arterial PaCO2 (mm Hg)   44.89 (12.07)   
4                   Arterial PaO2 (mm Hg)   77.61 (57.05)   
5                     Bicarbonate (mEq/L)    22.41 (4.89)   
6                       Bilirubin (mg/dL)      1.29 (2.3)   
7        Diastolic Blood Pressure (mm Hg)   57.39 (14.22)   
8         Systolic Blood Pressure (mm Hg)  100.28 (17.35)   
9                             BUN (mg/dL)   16.99 (15.45)   
10                        Calcium (mg/dL)     8.11 (0.84)   
11                Ionized Calcium (mg/dL)     4.68 (0.62)   
12                       Chloride (mEq/L)   110.92 (7.89)   
13                            CO2 (mEq/L)    21.61 (4.45)   
14                     Creatinine (mg/dL)     0.79 (1.01)   
15                               FiO2 (%)   48.24 (24.53)   
16                        Glucose (mg/dL)  142.85 (70.18)   
17                      Hemoglobin (g/dL)    10.65 (2.25)   
18                    Lactic Acid (mEq/L)      3.6 (3.49)   
19         Mean Arterial Pressure (mm Hg)   69.95 (13.85)   
20                 PaO2/FiO2 Ratio (mmHg)  150.16 (88.63)   
21                                     pH      7.31 (0.1)   
22                     Platelets (x10⁹/L)  225.2 (136.02)   
23                      Potassium (mEq/L)     3.95 (0.81)   
24                          PTT (seconds)   38.99 (16.13)   
25          Heart Rate (beats per minute)  126.99 (28.08)   
26                   Pupil Left Size (mm)     2.59 (1.08)   
27  Respiratory Rate (breaths per minute)   26.16 (10.07)   
28                         Sodium (mEq/L)    142.92 (7.5)   
29                               SpO2 (%)    97.67 (3.83)   
30                       Temperature (°C)     37.1 (0.78)   
31                           WBC (x10⁹/L)    12.08 (7.86)   

                                              Validation  \
          Median (Q1, Q3)      Min, Max        Mean (SD)   
0          2.7 (2.3, 3.1)      1.3, 4.9       2.76 (0.6)   
1          2.0 (0.0, 4.0)     0.0, 20.0      2.71 (3.08)   
2          5.0 (3.0, 7.0)     1.0, 26.0       5.6 (4.06)   
3      42.95 (36.4, 51.5)    17.0, 85.8    43.93 (12.21)   
4       58.0 (41.0, 90.0)   21.0, 383.0    72.64 (41.42)   
5       22.3 (19.6, 25.1)     4.8, 39.2     22.37 (4.81)   
6          0.5 (0.3, 1.0)     0.1, 16.5      0.85 (1.61)   
7       56.0 (47.0, 66.0)   31.0, 100.0     55.92 (14.4)   
8      99.0 (88.0, 112.0)   64.0, 150.0    99.79 (15.58)   
9        12.0 (8.0, 19.0)     2.0, 97.0    13.94 (10.98)   
10         8.1 (7.6, 8.6)     5.9, 10.7      8.31 (0.79)   
11         4.8 (4.3, 5.1)      2.7, 6.2      4.52 (0.74)   
12   110.0 (106.0, 115.0)   90.0, 137.0    110.71 (7.42)   
13      22.0 (19.0, 24.0)     5.0, 34.0     21.27 (4.37)   
14         0.5 (0.3, 0.8)    0.19, 11.1       0.56 (0.4)   
15      40.0 (30.0, 60.0)   21.0, 100.0     46.1 (22.67)   
16    122.0 (97.5, 163.5)   56.0, 602.0   143.01 (72.68)   
17      10.7 (9.1, 12.12)     4.5, 17.3     11.16 (2.19)   
18         2.3 (1.3, 4.5)     0.4, 23.0      4.26 (3.79)   
19      69.0 (60.0, 79.0)   42.0, 111.0     69.2 (13.83)   
20  125.5 (78.33, 208.12)  29.47, 390.0   184.63 (97.81)   
21      7.32 (7.26, 7.37)    6.95, 7.53       7.32 (0.1)   
22   206.0 (121.0, 305.0)   19.0, 749.0  249.07 (129.84)   
23         3.8 (3.4, 4.4)      2.4, 7.1      3.96 (0.77)   
24      34.3 (29.8, 42.1)   21.4, 140.8    37.45 (10.92)   
25   127.5 (107.0, 147.5)   58.0, 189.0    124.6 (28.63)   
26         2.0 (2.0, 3.0)      1.0, 6.0       2.58 (1.1)   
27      24.0 (19.5, 30.0)    10.0, 68.0     27.51 (9.42)   
28   141.0 (138.0, 146.0)  126.0, 170.0    141.62 (6.63)   
29     99.0 (97.0, 100.0)   73.0, 100.0      97.75 (2.4)   
30    37.0 (36.56, 37.61)  35.22, 39.28